### SVM

Implemantação do classificador utilizando SVM.

In [1]:
import nltk
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from string import ascii_lowercase

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report

from pymongo import MongoClient

%matplotlib inline

In [2]:
'''
Importação dos dados da base
'''
client = MongoClient()
client = MongoClient('localhost', 27800)
db = client.network_logs
db_unique_hostnames = db.unique_hostnames

df_hostnames = pd.DataFrame(list(db_unique_hostnames.find({})))

print(len(df_hostnames))
df_hostnames.head()

24208


,_id,catdesc,category,hostname,normalized_hostname,normalized_hostname_url,normalized_url,service,url
0,59a1e31988b032071ef8c495,Information Technology,Information Technology,sb.scorecardresearch.com,scorecardresearch,scorecardresearch,,HTTPS,/
1,59a1e31a88b032071ef8c496,Information Technology,Information Technology,test.com,test,testpartneruidUPcbedebee,partneruidUPcbedebee,HTTP,/?partner_uid=UP47c5b905-0e1d-11e7-b092-00163e...
2,59a1e31a88b032071ef8c497,Information Technology,Information Technology,nexus-websocket-b.intercom.io,nexuswebsocketbintercom,nexuswebsocketbintercom,,HTTPS,/
3,59a1e31a88b032071ef8c498,Malicious Websites,Malicious or Illegal,aorta.clickagy.com,aortaclickagy,aortaclickagypixelgifadvertiseridgtqxdqpglista...,pixelgifadvertiseridgtqxdqpglistaedjpnobncchcm...,HTTP,/pixel.gif?advertiser_id=1gt8qx97dqpg&list=11a...
4,59a1e31a88b032071ef8c499,Information Technology,Information Technology,usersync.videoamp.com,usersyncvideoamp,usersyncvideoampusersyncpartneridpartneruserid...,usersyncpartneridpartneruseridredirecthttpsync...,HTTP,/usersync?partner_id=5797361&partner_user_id=4...


In [3]:
'''
Criação do vocabulário
'''
keywords = [''.join(trig) for trig in itertools.product(ascii_lowercase, repeat = 3)]
len(keywords)

17576

### Vetorização e frequência dos termos

In [4]:
count_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3,3), vocabulary=keywords)
words_vector = count_vectorizer.transform(df_hostnames['normalized_hostname'])
tf_transformer = TfidfTransformer(use_idf=False).fit(words_vector)
urls_tf = tf_transformer.transform(words_vector)

### Separação do conjunto

In [5]:
labels = df_hostnames['category']
url_train,url_test,label_train,label_test = train_test_split(urls_tf, labels, test_size=0.3)

### Classificação e teste

In [6]:
#{'C': 4.4450000000000003, 'gamma': 1.0}
classifier_model = SVC(C=8.1111111111, gamma=3.5).fit(url_train, label_train)

In [7]:
predictions = classifier_model.predict(url_test)

### Relatório de classificação

In [8]:
print(classification_report(label_test, predictions))

                                 precision    recall  f1-score   support

                          Adult       0.00      0.00      0.00        22
                    Advertising       0.89      0.19      0.32       279
               Arts and Culture       1.00      0.06      0.12        49
                       Business       0.77      0.11      0.19       700
                Content Servers       0.95      0.59      0.72       376
                 Domain Parking       0.00      0.00      0.00        17
                      Education       0.93      0.06      0.12       225
                  Entertainment       0.50      0.01      0.02       128
       File Sharing and Storage       0.99      0.66      0.79       128
            Finance and Banking       1.00      0.05      0.09        63
Freeware and Software Downloads       1.00      0.02      0.04        55
                          Games       0.93      0.16      0.27       169
            Health and Wellness       1.00      0.

/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
17576 ** (1/2)